### Importation des packages

In [ ]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import json
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


### Récupération des liens sur la page principale

In [ ]:
url = "https://freeglisse.com/fr/12-ski-occasion"
reponse = requests.get(url)
html_content = reponse.text
soup = BeautifulSoup(html_content, "html.parser")
soup

In [ ]:
# afficher une page
def get_products():
	links_list = []
	url = 'https://freeglisse.com/fr/12-ski-occasion?page=1'
	response = requests.get(url)
	soup = BeautifulSoup(response.content, "html.parser")
	h2_tags = soup.find_all('h2', class_ = 'h3 product-title')
	for ski in h2_tags:
		product_links = [ski.a.get("href") for ski in h2_tags]
	links_list.extend(product_links)
	return links_list
get_products()


In [ ]:
def get_links():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/?page={}"
    current_page = 1

    links_list = []

    while True:
        url = base_url.format(current_page)

        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        h2_tags = soup.find_all("h2", {"class": "h3 product-title"})
        product_links = [h2_tag.a.get("href") for h2_tag in h2_tags]

        links_list.extend(product_links)

        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break

        current_page += 1

    return links_list

liste_lien = get_links()
print(liste_lien)
(len(liste_lien))

### Récupération des informations sur UNE page produit

In [ ]:
#Test de récupération sur une page produit
url_t = "https://freeglisse.com/fr/ski-occasion-femme-loisir/18890-430528-ski-occasion-rossignol-nova-6-fixations.html#/3-etat_du_materiel-qualite_a/891-taille_skis-149_cm"
reponse_t = requests.get(url_t)
html_content_t = reponse_t.text
soup_t = BeautifulSoup(html_content_t, "html.parser")

# pour obtenir la RÉFÉRENCE produit
ref_produit = (soup_t.find("div", {"class": "product-reference rb-tag-cate"}).text.strip()).replace("announcement Référence: ", "").strip()
print(ref_produit)
# pour obtenir le NOM du produit
nom_produit = soup_t.find_all("h1", {"class" : "h1 product-detail-name"})[0].get_text().strip()
print(nom_produit)
# pour obtenir la MARQUE du produit
marque_produit = soup_t.find("img", {"class" : "img img-thumbnail manufacturer-logo"}).get("alt", "")
print(marque_produit)
# pour obtenir le PRIX du produit
prix_produit = unidecode(soup_t.find_all("span", {"class" : "current-price-value"})[0].get_text().strip())
print(prix_produit)
# pour obtenir le TYPE DE PRATIQUE du produit
type_pratique = [soup_t.find_all("dd", {"class" : "value"})[0].get_text().strip()]
print(type_pratique)
# pour obtenir le TYPE D'UTILISATEUR du produit
utilisateur = [soup_t.find_all("dd", {"class" : "value"})[1].get_text().strip()]
print(utilisateur)
# pour obtenir le NIVEAU du produit
niveau = [soup_t.find_all("dd", {"class" : "value"})[2].get_text().strip()]
print(niveau)
# pour obtenir la COULEUR du produit
colori = [soup_t.find_all("dd", {"class" : "value"})[3].get_text().strip()]
print(colori)
# pour obtenir le NOMBRE D'AVIS du produit
nb_avis = soup_t.find("p", {"class" : "netreviews_subtitle"}).find("span", {"id": "reviewCount"}).text.strip()
print(nb_avis)
# pour obtenir la NOTE du produit
note_produit = soup_t.find("p", {"class" : "netreviews_note_generale"}).text.strip().split('/')[0]
print(note_produit)

#### Pour obtenir le nombre de produit disponible :

In [ ]:

# Initialiser le driver (ajustez pour votre propre navigateur)
driver = webdriver.Chrome()

# Ouvrir la page du produit
driver.get("https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/15483-167626-ski-de-fond-occasion-toutes-marques-fixation-vieux-sns.html#/882-taille_skis-140_cm/1768-etat_du_materiel-qualite_c")

# Attendre que la page soit complètement chargée
wait = WebDriverWait(driver, 10)

# Localiser le champ de quantité
quantity_input = wait.until(EC.visibility_of_element_located((By.ID, "quantity_wanted")))

# Variable pour stocker le maximum de produits en stock
max_stock = 1  # Commencer avec une quantité de 1

while True:
    # Défiler pour que l'élément de quantité soit visible
    driver.execute_script("arguments[0].scrollIntoView(true);", quantity_input)
    time.sleep(0.5)  # Petite pause pour s'assurer que l'élément est visible

    # Utiliser JavaScript pour définir la valeur du champ de quantité
    driver.execute_script(f"arguments[0].value = {max_stock};", quantity_input)
    
    # Simuler un événement 'change' sur le champ de quantité
    driver.execute_script("var event = new Event('change', { bubbles: true }); arguments[0].dispatchEvent(event);", quantity_input)
    
    # Attendre un peu pour que la page traite l'événement
    time.sleep(2)  # Ajustez cette durée si nécessaire

    # Rechercher le message de disponibilité à chaque itération
    try:
        # Localiser à nouveau le message de disponibilité
        availability_message = wait.until(
            EC.visibility_of_element_located((By.XPATH, "//span[@id='availability_message']"))
        )
        availability_text = availability_message.text
        print(f"Message de disponibilité : {availability_text}")

        # Vérifier si le stock est suffisant
        if "pas assez" in availability_text.lower():
            print(f"Stock maximum détecté avant l'erreur : {max_stock - 1}")
            break
    except Exception as e:
        print(f"Erreur lors de la vérification du message : {str(e)}")
        break
    
    # Si le message est valide, augmenter la quantité
    max_stock += 1

# Fermer le navigateur
driver.quit()

In [ ]:
def get_all_pages(base_url):
    urls = []
    current_page = 1
    
    while True:
        full_url = f"{base_url}{current_page}"
        response = requests.get(full_url)
        
        if response.status_code != 200:
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        urls.append(full_url)
        
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        
        current_page += 1
    
    return urls

qualiteA = get_all_pages('https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=')
qualiteB = get_all_pages('https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b?page=')
qualiteC = get_all_pages('https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c?page=')

In [ ]:
qualiteA

### Obtention de tous les liens disponibles sur le site


Afin de récupérer tous les liens des skis, nous allons faire une boucle par qualité (A,B et C). Il fautdrai également obtenir les liens en fonction des tailles (ce qui n'est pas encore fait ici)

#### Table qualité A

In [ ]:
url_a = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a"
reponse_a = requests.get(url_a)
html_content_a = reponse_a.text
soup_a = BeautifulSoup(html_content_a, "html.parser")

def table_qualite_a():
	base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page={}&order=product.price.desc"
	current_page = 1

	df_global_a = pd.DataFrame(columns=['nom', 'prix', 'lien'])

	while True:
		url = base_url.format(current_page)

		response = requests.get(url)
		if response.status_code != 200:
			print("Erreur lors de la requête HTTP")
			break

		soup = BeautifulSoup(response.text, 'html.parser')

		balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
		noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]

		prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]

		product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]

		data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
		df_current = pd.DataFrame(data)

		df_global_a = pd.concat([df_global_a, df_current], ignore_index=True)

		next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
		if not next_page_link:
			break

		current_page += 1
	return df_global_a

df_a = table_qualite_a()

df_a['Qualité'] = 'Qualité A'
df_a

#### Table qualité B

In [ ]:
url_b = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b"
reponse_b = requests.get(url_b)
html_content_b = reponse_b.text
soup_b = BeautifulSoup(html_content_b, "html.parser")

def table_qualite_b():
	base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b?page={}&order=product.price.desc"
	current_page = 1

	df_global_b = pd.DataFrame(columns=['nom', 'prix', 'lien'])

	while True:
		url = base_url.format(current_page)

		response = requests.get(url)
		if response.status_code != 200:
			print("Erreur lors de la requête HTTP")
			break

		soup = BeautifulSoup(response.text, 'html.parser')

		balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
		noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]

		prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]

		product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]

		data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
		df_current = pd.DataFrame(data)

		df_global_b = pd.concat([df_global_b, df_current], ignore_index=True)

		next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
		if not next_page_link:
			break

		current_page += 1
	return df_global_b

df_b = table_qualite_b()

df_b['Qualité'] = 'Qualité B'
df_b

#### Table qualité C

In [ ]:
url_c = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c"
reponse_c = requests.get(url_c)
html_content_c = reponse_c.text
soup_c = BeautifulSoup(html_content_c, "html.parser")

def table_qualite_c():
	base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c?page={}&order=product.price.desc"
	current_page = 1

	df_global_c = pd.DataFrame(columns=['nom', 'prix', 'lien'])

	while True:
		url = base_url.format(current_page)

		response = requests.get(url)
		if response.status_code != 200:
			print("Erreur lors de la requête HTTP")
			break

		soup = BeautifulSoup(response.text, 'html.parser')

		balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
		noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]

		prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]

		product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]

		data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
		df_current = pd.DataFrame(data)

		df_global_c = pd.concat([df_global_c, df_current], ignore_index=True)

		next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
		if not next_page_link:
			break

		current_page += 1
	return df_global_c

df_c = table_qualite_c()

df_c['Qualité'] = 'Qualité C'
df_c

#### MERGE A,B et C pour All_DF

In [ ]:
frames = [df_a, df_b, df_c]
all_df = pd.concat(frames)
display(all_df)

In [ ]:
all_df.groupby(['lien', 'Qualité']).count().reset_index().sort_values(by='nom', ascending=False).drop_duplicates(subset=['lien']).value_counts('Qualité')

In [ ]:
all_df

### Boucle V1 (SANS les stocks) pour obtenir toutes les informations sur toutes les pages 

In [ ]:
def TOUFIK():

    # CREATION DES LISTES
    nom_produit_list = []
    marque_produit_list = []
    prix_produit_list = []
    ref_produit_list = []
    type_pratique_list = []
    utilisateur_list = []
    niveau_list = []
    colori_list = []
    nb_avis_list = []
    note_produit_list = []
    availability_msg_list = []

    for link in all_df['lien']:
        url_p = link
        reponse_p = requests.get(url_p)
        html_content_p = reponse_p.text
        soup_p = BeautifulSoup(html_content_p, "html.parser")

        # PART 2 - récupérer les infos de la page produit
        # Le NOM du produit
        nom_produit = soup_p.find_all("h1", {"class": "h1 product-detail-name"})
        nom_produit_list.append(nom_produit[0].get_text().strip() if nom_produit else np.nan)

        # La MARQUE du produit
        marque_produit = soup_p.find("img", {"class": "img img-thumbnail manufacturer-logo"})
        marque_produit_list.append(marque_produit.get("alt", "") if marque_produit else np.nan)

        # Le PRIX du produit
        prix_produit = soup_p.find_all("span", {"class": "current-price-value"})
        prix_produit_list.append(unidecode(prix_produit[0].get_text().strip()) if prix_produit else np.nan)

        # La REF du produit
        ref_produit = soup_p.find("div", {"class": "product-reference rb-tag-cate"})
        ref_produit_list.append(
            (ref_produit.text.strip().replace("announcement Référence: ", "").strip()) if ref_produit else np.nan)

        # La DESCRIPTION du produit
        descrip_prod_1 = soup_p.find_all("dd", {"class": "value"})
        type_pratique_list.append(descrip_prod_1[0].get_text().strip() if descrip_prod_1 else 'Null')
        utilisateur_list.append(descrip_prod_1[1].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 1 else np.nan)
        niveau_list.append(descrip_prod_1[2].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 2 else np.nan)
        colori_list .append(descrip_prod_1[3].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 3 else np.nan)

        #Le NB d'avis
        nb_avis_element = soup_p.find("p", {"class": "netreviews_subtitle"})
        nb_avis = nb_avis_element.find("span", {"id": "reviewCount"}).text.strip() if nb_avis_element else np.nan
        nb_avis_list.append(nb_avis)

        # La NOTE du 5
        note_produit_element = soup_p.find("p", {"class": "netreviews_note_generale"})
        note_produit = note_produit_element.text.strip().split('/')[0] if note_produit_element else np.nan
        note_produit_list.append(note_produit)

        # La DISPONIBILITÉ du produit
        availability_msg = soup_p.find('span', {'id': 'availability_message'})
        availability_msg_list.append(availability_msg.get_text().strip() if availability_msg else np.nan)

    # APPEND AUX LISTES
    all_df['nom_produit'] = nom_produit_list
    all_df['marque_produit'] = marque_produit_list
    all_df['prix_produit'] = prix_produit_list
    all_df['ref_produit'] = ref_produit_list
    all_df['type_pratique'] = type_pratique_list
    all_df['utilisateur'] = utilisateur_list
    all_df['niveau'] = niveau_list
    all_df['colori'] = colori_list
    all_df['nb_avis'] = nb_avis_list
    all_df['note_produit'] = note_produit_list
    all_df['availability_msg'] = availability_msg_list
    
TOUFIK()
all_df

In [ ]:
all_df.drop_duplicates(inplace=True)

In [ ]:
all_df.shape

In [ ]:
all_df

In [ ]:
#CheakPoint
all_df.to_csv('/Users/yacinehamdi/Downloads/freegliss.csv')
#df = pd.read_csv("/Users/yacinehamdi/Downloads/freegliss.csv")

### Boucle V2 (AVEC les stocks) pour obtenir toutes les informations sur toutes les pages 

Cette boucle peut-être très longue (environ 50 heures pour 2000 articles)
Nous allons donc tester sur les 10 premiers articles

In [ ]:
#test sur 10 articles
test = df_a.sample(10)
test

In [ ]:
def TOUFIK():
    # CREATION DES LISTES
    nom_produit_list = []
    marque_produit_list = []
    prix_produit_list = []
    ref_produit_list = []
    type_pratique_list = []
    utilisateur_list = []
    niveau_list = []
    colori_list = []
    nb_avis_list = []
    note_produit_list = []
    availability_msg_list = []
    stock_list = []  # Liste pour le stock

    # Initialiser le WebDriver (spécifiez le chemin vers le chromedriver si nécessaire)
    driver = webdriver.Chrome()

    for link in test['lien']:
        url_p = link
        reponse_p = requests.get(url_p)
        html_content_p = reponse_p.text
        soup_p = BeautifulSoup(html_content_p, "html.parser")

        # PART 2 - récupérer les infos de la page produit
        # Le NOM du produit
        nom_produit = soup_p.find_all("h1", {"class": "h1 product-detail-name"})
        nom_produit_list.append(nom_produit[0].get_text().strip() if nom_produit else np.nan)

        # La MARQUE du produit
        marque_produit = soup_p.find("img", {"class": "img img-thumbnail manufacturer-logo"})
        marque_produit_list.append(marque_produit.get("alt", "") if marque_produit else np.nan)

        # Le PRIX du produit
        prix_produit = soup_p.find_all("span", {"class": "current-price-value"})
        prix_produit_list.append(unidecode(prix_produit[0].get_text().strip()) if prix_produit else np.nan)

        # La REF du produit
        ref_produit = soup_p.find("div", {"class": "product-reference rb-tag-cate"})
        ref_produit_list.append(
            (ref_produit.text.strip().replace("announcement Référence: ", "").strip()) if ref_produit else np.nan)

        # La DESCRIPTION du produit
        descrip_prod_1 = soup_p.find_all("dd", {"class": "value"})
        type_pratique_list.append(descrip_prod_1[0].get_text().strip() if descrip_prod_1 else 'Null')
        utilisateur_list.append(descrip_prod_1[1].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 1 else np.nan)
        niveau_list.append(descrip_prod_1[2].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 2 else np.nan)
        colori_list.append(descrip_prod_1[3].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 3 else np.nan)

        # Le NB d'avis
        nb_avis_element = soup_p.find("p", {"class": "netreviews_subtitle"})
        nb_avis = nb_avis_element.find("span", {"id": "reviewCount"}).text.strip() if nb_avis_element else np.nan
        nb_avis_list.append(nb_avis)

        # La NOTE du produit
        note_produit_element = soup_p.find("p", {"class": "netreviews_note_generale"})
        note_produit = note_produit_element.text.strip().split('/')[0] if note_produit_element else np.nan
        note_produit_list.append(note_produit)

        # Ouvrir la page avec Selenium pour récupérer le stock
        driver.get(url_p)

        # Attendre que la page soit complètement chargée
        wait = WebDriverWait(driver, 10)

        # Variable pour stocker le maximum de produits en stock
        max_stock = 1  # Commencer avec une quantité de 1
        stock_found = False  # Variable pour détecter si le stock est trouvé

        try:
            # Localiser le champ de quantité
            quantity_input = wait.until(EC.visibility_of_element_located((By.ID, "quantity_wanted")))

            while True:
                # Utiliser JavaScript pour définir la valeur du champ de quantité
                driver.execute_script(f"arguments[0].value = {max_stock};", quantity_input)
                
                # Simuler un événement 'change' sur le champ de quantité
                driver.execute_script("var event = new Event('change', { bubbles: true }); arguments[0].dispatchEvent(event);", quantity_input)
                        # Faire défiler vers le haut de la page
                driver.execute_script("window.scrollTo(1, 1);")
                time.sleep(1)  # Petite pause pour s'assurer que le scroll est terminé
                # Attendre un peu pour que la page traite l'événement
                time.sleep(2)  # Ajustez cette durée si nécessaire
                # Rechercher le message de disponibilité à chaque itération
                try:
                    # Localiser à nouveau le message de disponibilité
                    availability_message = wait.until(
                        EC.visibility_of_element_located((By.XPATH, "//span[@id='availability_message']"))
                    )
                    availability_text = availability_message.text
                    print(f"Message de disponibilité : {availability_text}")

                    # Vérifier si le stock est insuffisant
                    if "pas assez" in availability_text.lower():
                        stock_list.append(max_stock - 1)
                        stock_found = True
                        break

                except Exception as e:
                    print(f"Erreur lors de la vérification du message : {str(e)}")
                    break
                
                # Si le message est valide, augmenter la quantité
                max_stock += 1

                # Défiler pour que l'élément de quantité soit visible
                driver.execute_script("arguments[0].scrollIntoView(true);", quantity_input)
                time.sleep(0.5)  # Petite pause pour s'assurer que l'élément est visible

            # Si aucun stock n'est trouvé, ajouter np.nan
            if not stock_found:
                stock_list.append(np.nan)

        except Exception as e:
            print(f"Erreur lors de la vérification du stock : {e}")
            stock_list.append(np.nan)  # En cas d'erreur, on ajoute np.nan

    # APPEND AUX LISTES
    test['nom_produit'] = nom_produit_list
    test['marque_produit'] = marque_produit_list
    test['prix_produit'] = prix_produit_list
    test['ref_produit'] = ref_produit_list
    test['type_pratique'] = type_pratique_list
    test['utilisateur'] = utilisateur_list
    test['niveau'] = niveau_list
    test['colori'] = colori_list
    test['nb_avis'] = nb_avis_list
    test['note_produit'] = note_produit_list
    #test['availability_msg'] = availability_msg_list
    test['stock'] = stock_list  # Ajout des stocks à la DataFrame

    # Fermer le WebDriver à la fin
    driver.quit()

TOUFIK()
display(test)